In [0]:
import nibabel as nib
import glob
import os
import numpy as np
#from utils import cropVolume

In [0]:
import numpy as np # this import needs to be here bc of ucVisualizeResults

def cropVolume(img, data=False):
    '''
    Helper function to remove the redundant black area from the 3D volume
    :param img: 3D volume
    :param data: Nib allows you to access 3D volume data using get_data(). 
    If you have already used it before calling this function, then it is false
    :return: returns the crop positions across 3 axes (height, width, channel)
    '''
    if not data:
       img = img.get_data()
    sum_array = []

    for ch in range(img.shape[2]): # dim 3
        values, indexes = np.where(img[:, :, ch] > 0) # values > 0 are not black??
        sum_val = sum(values) # single value
        sum_array.append(sum_val) # list of 3 elmts
    ch_s = np.nonzero(sum_array)[0][0] # first choose array/list in list of arrays/lists, 
                                       # then choose index in the list
    ch_e = np.nonzero(sum_array)[0][-1] 
    sum_array = []
    for height in range(img.shape[0]): # let's say height is the number of rows, 
                                       # or zero_th dim in .shape
        values, indexes = np.where(img[height, :, :] > 0)
        sum_val = sum(values)
        sum_array.append(sum_val)
    hi_s = np.nonzero(sum_array)[0][0]
    hi_e = np.nonzero(sum_array)[0][-1]
    sum_array = []
    for width in range(img.shape[1]):
        values, indexes = np.where(img[:, width, :] > 0)
        sum_val = sum(values)
        sum_array.append(sum_val)
    wi_s = np.nonzero(sum_array)[0][0]
    wi_e = np.nonzero(sum_array)[0][-1]

    return hi_s, hi_e, wi_s, wi_e, ch_s, ch_e # s is start and e is end
# REMEMBER to Change all Height and Width Objects # 

In [0]:
import numpy as np

def cropVolumes(img1, img2, img3, img4):
    '''
        This function crops the 4 volumes that BRATS dataset provides
        :param img1: Volume 1
        :param img2: Volume 2
        :param img3: Volume 3
        :param img4: Volume 4
        :return: maximum dimensions across three dimensions
    '''
    hi_s1, hi_e1, wi_s1, wi_e1, ch_s1, ch_e1 = cropVolume(img1, True)#.............................
    hi_s2, hi_e2, wi_s2, wi_e2, ch_s2, ch_e2  = cropVolume(img2, True)
    hi_s3, hi_e3, wi_s3, wi_e3, ch_s3, ch_e3 = cropVolume(img3, True)
    hi_s4, hi_e4, wi_s4, wi_e4, ch_s4, ch_e4 = cropVolume(img4, True)

    ch_st = min(ch_s1, ch_s2, ch_s3, ch_s4)#.............................
    ch_en = max(ch_e1, ch_e2, ch_e3, ch_e4)
    wi_st = min(wi_s1, wi_s2, wi_s3, wi_s4)
    wi_en = max(wi_e1, wi_e2, wi_e3, wi_e4)
    hi_st = min(hi_s1, hi_s2, hi_s3, hi_s4)
    hi_en = max(hi_e1, hi_e2, hi_e3, hi_e4)
    return hi_st, hi_en, wi_st, wi_en, ch_st, ch_en

In [0]:
def cropAndSaveVolumes(imgLoc, dataset_name):
    t1_loc = imgLoc.replace('flair', 't1')#........................................
    t1ce_loc = imgLoc.replace('flair', 't1ce')
    t2_loc = imgLoc.replace('flair', 't2')
    gth_loc = imgLoc.replace('flair', 'seg')


    img_flair = nib.load(imgLoc)
    affine_flair = img_flair.affine # affine transformation ??................................................
    header_flair = img_flair.header

    img_t1 = nib.load(t1_loc)
    affine_t1 = img_t1.affine
    header_t1 = img_t1.header

    img_t1ce = nib.load(t1ce_loc)
    affine_t1ce = img_t1ce.affine
    header_t1ce = img_t1ce.header

    img_t2 = nib.load(t2_loc)
    affine_t2 = img_t2.affine
    header_t2 = img_t2.header

    gth = nib.load(gth_loc)
    affine5 = gth.affine
    header5 = gth.header

    img_flair = img_flair.get_data()
    img_t1 = img_t1.get_data()
    img_t1ce = img_t1ce.get_data()
    img_t2 = img_t2.get_data()
    gth = gth.get_data()


    # Crop the volumes
    # First, identify the max dimensions and then crop
    hi_st, hi_en, wi_st, wi_en, ch_st, ch_en = cropVolumes(img_flair, img_t1, img_t1ce, img_t2)
    img_flair = img_flair[hi_st:hi_en, wi_st:wi_en, ch_st:ch_en] #.....................................
    img_t1 = img_t1[hi_st:hi_en, wi_st:wi_en, ch_st:ch_en]
    img_t1ce = img_t1ce[hi_st:hi_en, wi_st:wi_en, ch_st:ch_en]
    img_t2 = img_t2[hi_st:hi_en, wi_st:wi_en, ch_st:ch_en]
    gth = gth[hi_st:hi_en, wi_st:wi_en, ch_st:ch_en]

    name = imgLoc.replace(imgLoc.split('/')[-1], '').replace(dataset_name, dataset_name + '_preprocess')
    # fix dataset_name

    # create the directories if they do not exist
    if not os.path.isdir(name):
        os.makedirs(name)

    # save the cropped volumes
    img_flair_cropped = nib.Nifti1Image(img_flair, affine_flair, header_flair)#.............................................
    nib.save(img_flair_cropped, name + os.sep + imgLoc.split('/')[-1])

    img_t1_cropped = nib.Nifti1Image(img_t1, affine_t1, header_t1)
    nib.save(img_t1_cropped, name + os.sep + t1_loc.split('/')[-1])

    img_t1ce_cropped = nib.Nifti1Image(img_t1ce, affine_t1ce, header_t1ce)
    nib.save(img_t1ce_cropped, name + os.sep + t1ce_loc.split('/')[-1])

    img_t2_cropped = nib.Nifti1Image(img_t2, affine_t2, header_t2)
    nib.save(img_t2_cropped, name + os.sep + t2_loc.split('/')[-1])

    gth_cropped = nib.Nifti1Image(gth, affine5, header5)
    nib.save(gth_cropped, name + os.sep + gth_loc.split('/')[-1])

In [0]:
if __name__ == '__main__':
    #dims_0 = []
    #dims_1 = []
    #dims_2 = []
    #data_dir = './data/'
    data_dir = '/content/drive/My Drive/MICCAI_BraTS_2019_Data_Training'
    ##sec_dir = '/content/drive/My Drive/backup_preprocess'
    #dataset_name = 'original_brats18'
    for src_folder in ['LGG', 'HGG']:
        #src_dir = data_dir + os.sep + dataset_name + os.sep + src_folder + os.sep
        src_dir = data_dir + os.sep + src_folder + os.sep

        folder_list = glob.glob(src_dir + '*')
        for folder in folder_list:
            files = glob.glob(folder + '/*_flair.nii.gz')
            for file in files:
                #cropAndSaveVolumes(file, dataset_name)
                cropAndSaveVolumes(file, sec_dir)
                print(file)

/content/drive/My Drive/MICCAI_BraTS_2019_Data_Training/LGG/BraTS19_TCIA13_634_1/BraTS19_TCIA13_634_1_flair.nii.gz
/content/drive/My Drive/MICCAI_BraTS_2019_Data_Training/LGG/BraTS19_TCIA13_650_1/BraTS19_TCIA13_650_1_flair.nii.gz
/content/drive/My Drive/MICCAI_BraTS_2019_Data_Training/LGG/BraTS19_TMC_09043_1/BraTS19_TMC_09043_1_flair.nii.gz
/content/drive/My Drive/MICCAI_BraTS_2019_Data_Training/LGG/BraTS19_TCIA13_654_1/BraTS19_TCIA13_654_1_flair.nii.gz
/content/drive/My Drive/MICCAI_BraTS_2019_Data_Training/LGG/BraTS19_TCIA13_653_1/BraTS19_TCIA13_653_1_flair.nii.gz
/content/drive/My Drive/MICCAI_BraTS_2019_Data_Training/LGG/BraTS19_TCIA13_633_1/BraTS19_TCIA13_633_1_flair.nii.gz
/content/drive/My Drive/MICCAI_BraTS_2019_Data_Training/LGG/BraTS19_TCIA13_642_1/BraTS19_TCIA13_642_1_flair.nii.gz
/content/drive/My Drive/MICCAI_BraTS_2019_Data_Training/LGG/BraTS19_TCIA13_645_1/BraTS19_TCIA13_645_1_flair.nii.gz
/content/drive/My Drive/MICCAI_BraTS_2019_Data_Training/LGG/BraTS19_TCIA13_615_1/B

KeyboardInterrupt: ignored